# Text Quality Assessment

In [1]:
import torch
from tqdm import tqdm
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from datasets import Dataset

from discover.container import DiscoverContainer
from discover.assets.idgen import AssetIDGen
from discover.core.flow import DataPrepStageDef, PhaseDef

In [2]:
container = DiscoverContainer()
container.init_resources()
container.wire(
    modules=[
        "discover.flow.data_prep.stage",
        "discover.analysis.base",
    ],
)

## Create HuggingFace Dataset

In [3]:
import distributed


idgen = AssetIDGen()
asset_id = idgen.get_asset_id(
    asset_type="dataset",
    phase=PhaseDef.DATAPREP,
    stage=DataPrepStageDef.FEATURE,
    name="review",
)
# Instantiate the repository
repo = container.repo.dataset_repo()
# Load the dataset from the repository
dataset = repo.get(asset_id, distributed=False)
# Create Huggingface Dataset object
dataset = Dataset.from_pandas(dataset.content)

AttributeError: 'list' object has no attribute 'is_unique'

## Load GPT-2

In [ ]:
device = "cuda"
model_id = "openai-community/gpt2-large"
model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

## Encode Dataset

In [ ]:
encodings = tokenizer("\n\n".join(dataset["content"]), return_tensors="pt")

In [ ]:
encodings[0]

## Perplexity on Fixed Length Model

In [ ]:
max_length = model.config.n_positions
stride = 512
seq_len = encodings.input_ids.size(1)

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())